<a href="https://colab.research.google.com/github/inspire-lab/CyberAI-labs/blob/main/category-PrivateAI/Differential-privacy-PATE/PATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Differentially Private MNIST Classification using PATE framework in PyTorch

This lab will guide you through implementing a differentially private MNIST classifier using PyTorch.
You will create an ensemble of teacher models and train a student model based on their aggregated predictions.

### Objectives
1. Load and preprocess the MNIST dataset.
2. Set up and train teacher models for privacy-preserving training.
3. Train a student model using labels generated with differential privacy techniques.
4. Evaluate the accuracy of the student model.

## Overview of PATE

The PATE (Private Aggregation of Teacher Ensembles) framework is a differential privacy technique designed to train machine learning models while protecting individual data privacy. It uses an ensemble of "teacher" models, each trained on separate subsets of data, to guide a "student" model. The student learns from the aggregated outputs (votes) of the teachers on input queries, with noise added to ensure differential privacy. By relying on consensus from multiple teachers, PATE reduces the impact of any one individual’s data on the student model, thus preserving privacy.


## Step 1: Import Necessary Libraries

In this step, you'll import essential libraries, including PyTorch and Torchvision, for building and managing the neural network and handling the dataset.

Run the following code:


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)  # Display options for output



## Step 2: Load and Prepare Data

This step loads and preprocesses the MNIST dataset. The dataset is divided into subsets, each assigned to a separate teacher model for privacy.

1. Define the number of teacher models.
2. Load the MNIST training data.
3. Split the data for each teacher model.

Run the following code:


In [ ]:

train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

teachers_num = 100  # Define the number of teachers
teachers_batch_size = 30  # Teachers batch size
data_size = len(train_set) // teachers_num  # size of dataset for each teacher

teachers_set = torch.utils.data.random_split(train_set, [data_size] * teachers_num)



## Step 3: Define the Teacher Model Network

This neural network model will be used to train each teacher. It has two convolutional layers and two fully connected layer and an output layer.

Run the following code:


In [ ]:

class Network(nn.Module):
    # Network used to train the Teachers

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)  # Flattened features
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)

        return t



## Step 4: Helper Classes for Managing Training Runs

These classes help manage training runs, track metrics, and log results.
- **RunBuilder** generates parameter combinations for training runs.
- **RunManager** tracks metrics like loss and accuracy and saves the results.

Run the following code:


In [ ]:

from itertools import product
from collections import namedtuple, OrderedDict
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import json

class RunBuilder():
    @staticmethod
    def get_runs(params):
        # Generates all parameter combinations.
        Run = namedtuple('Run', params.keys())
        return [Run(*values) for values in product(*params.values())]

class RunManager():
    # Manages each run by tracking loss, accuracy, and saving results.

    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self, run, network, loader):
        self.run_params = run
        self.run_count += 1
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        self.tb.add_image('images', grid)
        self.tb.add_graph(self.network, images)

    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy

        for k, v in self.run_params._asdict().items():
            results[k] = v
        self.run_data.append(results)
        clear_output(wait=True)
        display(pd.DataFrame.from_dict(self.run_data))

    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += preds.argmax(dim=1).eq(labels).sum().item()

    def save(self, file_name):
        pd.DataFrame.from_dict(self.run_data).to_csv(f'{file_name}.csv')
        with open(f'{file_name}.json', 'w') as f:
            json.dump(self.run_data, f, indent=4)



## Step 5: Training the Teacher Models

Using the helper classes, we now train an ensemble of teacher models. Each teacher model is trained on a unique subset of data, helping maintain differential privacy.

Run the following code to train the teacher models:


In [ ]:

from collections import OrderedDict

params = OrderedDict(
    lr = [.01],  # Learning rate
    batch_size = [teachers_batch_size],
    teacher = list(range(0, teachers_num))
)

m = RunManager()
teachers = []  # List of Aggregated teachers (Curator's Model)

for run in RunBuilder.get_runs(params):
    network = Network()
    teacher_loader =

#####################
#Your code goes here
#####################
    optimizer =

#####################
#Your code goes here
#####################

    m.begin_run(run, network, teacher_loader)

    for epoch in range(10):  # Train each teacher for 10 epochs
        m.begin_epoch()

        for batch in teacher_loader:
            images, labels = batch  # Load a batch of images and labels
            preds = network(images)  # Make predictions
            loss = F.cross_entropy(preds, labels)  # Calculate the loss
            optimizer.zero_grad()  # Zero the gradients
            loss.backward()  # Backpropagate
            optimizer.step()  # Update weights

            m.track_loss(loss)
            m.track_num_correct(preds, labels)

        m.end_epoch()

    m.end_run()
    teachers.append(network)  # Append each trained teacher model

m.save('results_teacher')  # Save results as CSV and JSON


,run,epoch,loss,accuracy,lr,batch_size,teacher
0,1,1,2.130584,0.185000,0.01,30,0
1,1,2,0.983092,0.680000,0.01,30,0
2,1,3,0.590239,0.821667,0.01,30,0
3,1,4,0.426030,0.870000,0.01,30,0
4,1,5,0.299171,0.921667,0.01,30,0
...,...,...,...,...,...,...,...
995,100,6,0.116958,0.951667,0.01,30,99
996,100,7,0.101168,0.975000,0.01,30,99
997,100,8,0.069900,0.975000,0.01,30,99
998,100,9,0.055120,0.983333,0.01,30,99



## Step 6: Creating and Preparing Public Dataset

Here, we load and split the MNIST test data to create a public dataset for training the student model.

Run the following code:


In [ ]:

test_set = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

student_train_set, student_test_set = torch.utils.data.random_split(test_set, [9000, 1000])  # Splits into train and test sets

student_train_loader = torch.utils.data.DataLoader(student_train_set, batch_size=100)
student_test_loader = torch.utils.data.DataLoader(student_test_set, batch_size=100)



## Step 7: Aggregating Predictions with Differential Privacy

We use Laplace noise to create labels for the student model based on the teacher models' predictions. Run the code below to aggregate predictions using the Laplace mechanism.

Run the following code:


In [ ]:
# PREDICT FUNCTION

def predict(network_model, dataloader):
    """
    This function predicts labels for a dataset.

    :param network_model:  the network model which is used for prediction
    :param dataloader DataLoader: Data input for the prediction.
    :return: tensor with the probabilities (ps) of the prediction model.
    :rtype: rank-1 tensor of size the size of the dataloader.
    """
    outputs = torch.zeros(0, dtype=torch.long)

    for images, labels in dataloader:
        output = network_model.forward(images)
        output = F.softmax(output, dim=1)
        ps = torch.argmax(output, dim=1)
        outputs = torch.cat((outputs, ps))

    return outputs

We will obtain a $(0.1, 0)-$differential private learning algorithm. Run the code below.

In [ ]:

import numpy as np

epsilon = 0.1

def aggregated_teacher_Laplace(teachers, dataLoader, epsilon):
    preds_teachers = torch.zeros((len(teachers), len(dataLoader.dataset)), dtype=torch.long)

    for i, teacher in enumerate(teachers):
        results = predict(teacher, dataLoader)
        preds_teachers[i] = results

    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds_teachers):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)

        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)

    return preds_teachers, labels

preds_teacher, student_labels = aggregated_teacher_Laplace(teachers, student_train_loader, epsilon)


<ipython-input-18-8a18c5a79541>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_counts[i] += np.random.laplace(0, beta, 1)



## Step 8: Training the Student Model

Using the differentially private labels generated from the teacher ensemble, we train the student model.

Run the following code:


In [ ]:

# Relabeling the training data for the student model
train_processing = torch.utils.data.DataLoader(student_train_set, batch_size=len(student_train_set))
student_images, _ = next(iter(train_processing))

student_tensor = torch.as_tensor(student_labels)
tensor_data = torch.utils.data.TensorDataset(student_images, student_tensor)
new_loader = torch.utils.data.DataLoader(tensor_data, batch_size=100)

params = OrderedDict(
    lr = [.01],
    batch_size = [30]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    student = Network()
    optimizer = optim.Adam(student.parameters(), lr=run.lr)
    m.begin_run(run, student, new_loader)

    for epoch in range(10):  # Train for 10 epochs
        m.begin_epoch()
        for batch in new_loader:
            images, labels = batch
            preds = student(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            m.track_loss(loss)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()

m.save('results_student')


,run,epoch,loss,accuracy,lr,batch_size
0,1,1,0.698789,0.764333,0.01,30
1,1,2,0.182948,0.946222,0.01,30
2,1,3,0.149342,0.956444,0.01,30
3,1,4,0.122329,0.964222,0.01,30
4,1,5,0.103402,0.969444,0.01,30
5,1,6,0.115458,0.968444,0.01,30
6,1,7,0.103695,0.970556,0.01,30
7,1,8,0.103005,0.970556,0.01,30
8,1,9,0.100579,0.970333,0.01,30
9,1,10,0.088807,0.974000,0.01,30



## Step 9: Final Evaluation of the Student Model

Evaluate the accuracy of the student model on the test dataset.

Run the following code:


In [ ]:

correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculations for inference
    for data in student_test_loader:
        images, labels = data
        outputs = student(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the student network on the test images: %d %%' % (
    100 * correct / total))  # Display accuracy as a percentage


Accuracy of the student network on the test images: 93 %


## References
---
1. [https://github.com/jcubit/differential-privacy/](https://github.com/jcubit/differential-privacy/)
2. https://arxiv.org/abs/1610.05755
3. https://arxiv.org/abs/1802.08908
